In [143]:
import zipfile
import os
import zipfile
from inference import get_model
from PIL import Image
import roboflow
import supervision as sv
import numpy as np
import cv2
import json
from dotenv import load_dotenv


In [145]:
load_dotenv()
api_key = os.getenv('ROBOFLOW_API_KEY_TEST')
project_name = os.getenv('ROBOFLOW_PROJECT_NAME')
print(project_name)

None


In [122]:
model = get_model("wigglebin/1")

In [123]:
print(model.class_names)

['fly', 'fly-larva', 'worm']


In [124]:
# Authenticate with Roboflow
rf = roboflow.Roboflow(api_key=api_key)
project = rf.workspace().project(project_name)

def send_to_roboflow(annotation_path, image_path, batch_name):
    # Upload image and annotations to Roboflow
    project.upload(image_path, annotation_path=annotation_path, batch_name=batch_name)

loading Roboflow workspace...
loading Roboflow project...


TypeError: argument of type 'NoneType' is not iterable

In [100]:
def extract_zip(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def analyze_image(image):
    results = model.infer(image)[0]
    return results

def get_model_results(image_path):
    results = model.infer(image=image_path)[0]

    return results

def draw_boxes(results, image_path, output_image_path):
    image = Image.open(image_path)
    image_np = np.array(image)  # Convert PIL image to NumPy array
    image_bgr = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)  # Convert RGB to BGR

    # Convert the inference results to Supervisely format
    detections = sv.Detections.from_inference(results)

    # Create supervision annotators
    bounding_box_annotator = sv.BoxAnnotator()  # Set the color for bounding boxes
    label_annotator = sv.LabelAnnotator()  # Set the color for labels

    labels = [
        f"{model.class_names[class_id]} {confidence:.2f}"
        for class_id, confidence in zip(detections.class_id, detections.confidence)
    ]

    # Annotate the image with our inference results
    annotated_image_rgb = bounding_box_annotator.annotate(scene=image_bgr, detections=detections)
    annotated_image_bgr = label_annotator.annotate(scene=annotated_image_rgb, detections=detections, labels=labels)

        # Convert the annotated image back to RGB format
    annotated_image_rgb = cv2.cvtColor(annotated_image_bgr, cv2.COLOR_BGR2RGB)
    annotated_image_pil = Image.fromarray(annotated_image_rgb)  # Convert NumPy array back to PIL image

    # Save the annotated image using PIL
    annotated_image_pil.save(output_image_path)

def format_annotations(predictions, image_file_name):
    data = []
    annotations = []
    for prediction in predictions:
        annotation = {
            "label": prediction.class_name,
            "coordinates": {
                "x": prediction.x,
                "y": prediction.y,
                "width": prediction.width,
                "height": prediction.height
            }
        }
        annotations.append(annotation)

        data.append({
            "image": image_file_name,
            "annotations": annotations
        })
    return data

def save_annotations(predictions, image_path, annotations_dir):
    image_file_name = os.path.basename(image_path)
    annotations = format_annotations(predictions, image_file_name)

    annotation_file_name = f"{os.path.splitext(image_file_name)[0]}.json"
    annotation_path = os.path.join(annotations_dir, annotation_file_name)
    with open(annotation_path, 'w') as f:
        json.dump(annotations, f)
    return annotation_path

In [116]:
# weeks = ['2024-33', '2024-35', '2024-37', '2024-39', '2024-41']
week = '2024-30'
zip_path = f'downloads/weekly/{week}.zip'
extraction_dir = f'extracted/weekly/{week}'
predictions_dir = f'predictions/weekly/{week}'
annotations_dir = f'annotations/weekly/{week}'

os.makedirs(predictions_dir, exist_ok=True)
os.makedirs(annotations_dir, exist_ok=True)

In [117]:
extract_zip(zip_path, extraction_dir)

## Draw predictions on images and send to Roboflow

In [118]:
for root, dirs, files in os.walk(extraction_dir):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(root, file)
            results = get_model_results(image_path)
            draw_boxes(
                results,
                image_path, 
                os.path.join(predictions_dir, file)
            )
            annotation_path = save_annotations(results.predictions, image_path, annotations_dir)
            send_to_roboflow(annotation_path, image_path, week)
            